# Generating ISBNs and Metadata

In [1]:
import re
import pandas as pd
import numpy as np

from CSVtoDF import CSVtoDF

I will use personally defined module `CSVtoDF`, which will temporary open csv file with with statement and only load manually picked columns and drop the rest and close the file.

In [2]:
with CSVtoDF('bestsellers_raw.csv') as df:
    df['isbn13'] = df['primary_isbn13']
    df['title1'] = df['title'].str.lower()
    df['author1'] = df['author'].str.lower()
    df['weeks'] = df['weeks_on_list']
df.head()

,isbn13,title1,author1,weeks
0,9781616200817,water for elephants,sara gruen,12
1,9781101513781,chasing fire,nora roberts,1
2,9780759514713,the lincoln lawyer,michael connelly,6
3,9780316069380,the fifth witness,michael connelly,2
4,9780425232200,the help,kathryn stockett,9


Actually we don't need ISBN10 as ISBN13 can better do the job.

We drop the `isbn` column and keep the remaining 3.

In [3]:
df = df[['isbn13', 'title1', 'author1', 'weeks']]
print()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8555 entries, 0 to 8554
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   isbn13   8552 non-null   object
 1   title1   8555 non-null   object
 2   author1  8555 non-null   object
 3   weeks    8555 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 267.5+ KB


We see that there are some _Null_ values in our dataframe.
But at this moment we are only interested with ISBN13 column, as the values found in that column will be used as a indexes to connect to the goodreads webpage for scraping.

In [4]:
df1 = df[df['isbn13'].isna()]
df1

,isbn13,title1,author1,weeks
143,NaN,summer secrets,barbara freethy,1
169,NaN,summer secrets,barbara freethy,2
196,NaN,summer secrets,barbara freethy,0


Fortunatelly, Summer Secrets by Barbara Freethy is the only book which has no ISBN13 information.
I will simply replace the empty value with ISBN number.

In [5]:
df['isbn13'].replace(np.NaN, 'B003K15AKQ', inplace=True)
print()
print(df.shape)
df.info()


(8555, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8555 entries, 0 to 8554
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   isbn13   8555 non-null   object
 1   title1   8555 non-null   object
 2   author1  8555 non-null   object
 3   weeks    8555 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 267.5+ KB


Next we have to see at the duplicate values in our dataframe.

First I combine Author's name, Title and ISBN13 number into one column. This way we will check for absolute duplicates (books that are exactly same editions, otherwise even if the same book is published with different cover or as revised version its ISBN will be changed).

In [6]:
df['author_title_isbn13'] = df['author1'] + ' ' + df['title1'] + ' ' + df['isbn13']
print()
df['author_title_isbn13'].head()

0         sara gruen water for elephants 9781616200817
1              nora roberts chasing fire 9781101513781
2    michael connelly the lincoln lawyer 9780759514713
3     michael connelly the fifth witness 9780316069380
4              kathryn stockett the help 9780425232200
Name: author_title_isbn13, dtype: object

In [7]:
dups = df.pivot_table(index=['author_title_isbn13'], aggfunc='size')
print()
print(dups.sort_values(ascending=False))


author_title_isbn13
gillian flynn gone girl 9780307588388               112
george rr martin a game of thrones 9780553897845     77
kristin hannah the nightingale 9781466850606         74
e l james fifty shades of grey 9781612130293         71
e l james fifty shades darker 9781612130590          70
                                                   ... 
kristen ashley walk through fire 9781455533244        1
kristen ashley the will A00B00HYIF9FW                 1
kristen ashley the slow burn A00B07P1HZQDH            1
kristen ashley the promise A00B00JXW6GFE              1
jo nesbo the son 9780385351386                        1
Length: 3084, dtype: int64


Now we can observe that Gone Girl and GOT and some other books appear several times in out df. That is because as mentioned earlier some books have been on the bestseller list for tens of weeks and their data came along with each week they have been featured.

We will drop those duplicates using `pandas` `drop_duplicates`.

In [8]:
df.drop_duplicates('author_title_isbn13', keep='last', ignore_index=True, inplace=True)
df = df[['title1', 'author1', 'isbn13']]
df.shape

(3084, 3)

We also observed in a previous cell that some books have ASIN code instead of ISBN, this can be issue as Goodreads can't identify books based on ASIN. So I'll filter them out as well.

In [9]:
aisbn = []
i13 = []

for i in list(df['isbn13']):
    if re.search(r'^[\dB]+', i):
        i13.append(i)
    else:
        aisbn.append(i)
        
print(len(aisbn))
print(len(i13))

149
2935


In [10]:
df_final = df[df['isbn13'].str.contains(r'^[\dB]+')]
df_final.shape

(2935, 3)

We are down to wooping 2935 entries, but this are original titles which can be used for further analysis.

We see that there are no more `NaN` values in `isbn13` column and no more duplicates, thus we can proceed and extract it as a list for web scraping.